In [1]:
import pickle
import pandas as pd
import multiprocessing as multi
from tqdm.auto import tqdm
from joblib import Memory
from pymatgen.core import Composition

from htvs_module.htvs_mods import VirtualEntry, get_window_range

In [2]:
df = pd.read_csv('./data/03_3_after_band_gap_screening.csv')
df

,composition,roost_ensemble_mean,e_hull_from_roost,roost_band_gap,pretty_formula
0,Na1Li6S4,-1.361550,0.066127,3.103082,Na(Li3S2)2
1,Li17Se6S5,-1.418221,-0.064147,2.161156,Li17Se6S5
2,Li1Se2S1,-0.717415,-0.067441,2.719070,LiSe2S
3,Na5Li1S2,-0.968061,0.012693,2.516691,Na5LiS2
4,Li3Hg8S5,-0.395427,0.193329,0.854263,Li3Hg8S5
...,...,...,...,...,...
301,Li2Sn15S19,-0.768460,0.005445,0.757977,Li2Sn15S19
302,Li6Ga1S4,-1.404617,-0.001322,2.905212,Li6GaS4
303,Li17Pb6S5,-0.763350,0.172872,0.511936,Li17Pb6S5
304,Ba15Li2S19,-2.013019,0.103946,2.870701,Ba15Li2S19


# Get stability window

In [3]:
unique_formula_df = df[['pretty_formula','composition']]
unique_formula_df

,pretty_formula,composition
0,Na(Li3S2)2,Na1Li6S4
1,Li17Se6S5,Li17Se6S5
2,LiSe2S,Li1Se2S1
3,Na5LiS2,Na5Li1S2
4,Li3Hg8S5,Li3Hg8S5
...,...,...
301,Li2Sn15S19,Li2Sn15S19
302,Li6GaS4,Li6Ga1S4
303,Li17Pb6S5,Li17Pb6S5
304,Ba15Li2S19,Ba15Li2S19


In [4]:
num_cores = multi.cpu_count() # 8
comp_list = unique_formula_df['pretty_formula']

memory = Memory('./data/stability_multiprocesser',verbose=0)
calc_stability_cached = memory.cache(get_window_range)

with multi.Pool(num_cores) as pool:
    stability = list(tqdm(pool.imap(calc_stability_cached, comp_list),total = len(comp_list)))

stability_df = pd.DataFrame(stability,columns=['lower_bound','upper_bound','total_range'])
stability_df = pd.concat([unique_formula_df,stability_df],axis=1)
stability_df

  0%|          | 0/306 [00:00<?, ?it/s]

,pretty_formula,composition,lower_bound,upper_bound,total_range
0,Na(Li3S2)2,Na1Li6S4,1.960835,1.960835e+00,1.499999e-08
1,Li17Se6S5,Li17Se6S5,2.032476,2.032476e+00,3.555564e-09
2,LiSe2S,Li1Se2S1,2.126687,2.126687e+00,1.368421e-08
3,Na5LiS2,Na5Li1S2,0.417076,4.170761e-01,2.000001e-08
4,Li3Hg8S5,Li3Hg8S5,1.876588,1.876588e+00,4.761906e-09
...,...,...,...,...,...
301,Li2Sn15S19,Li2Sn15S19,1.959325,1.959325e+00,8.000009e-09
302,Li6GaS4,Li6Ga1S4,1.304365,1.304365e+00,1.499999e-08
303,Li17Pb6S5,Li17Pb6S5,0.395109,3.951091e-01,1.111107e-08
304,Ba15Li2S19,Ba15Li2S19,1.955832,1.955832e+00,6.666638e-09


In [5]:
stability_df.to_csv('./data/04_1_stability_window.csv',index=False)
stability_df=pd.read_csv('./data/04_1_stability_window.csv')

# Cathode voltage > 1V & Total stability window range > 0.001 (example)

In [6]:
screened_by_stability_window_df = stability_df[(stability_df['upper_bound']>1)&(stability_df['total_range']>0.001)]

# Stability window screening result

In [7]:
df

,composition,roost_ensemble_mean,e_hull_from_roost,roost_band_gap,pretty_formula
0,Na1Li6S4,-1.361550,0.066127,3.103082,Na(Li3S2)2
1,Li17Se6S5,-1.418221,-0.064147,2.161156,Li17Se6S5
2,Li1Se2S1,-0.717415,-0.067441,2.719070,LiSe2S
3,Na5Li1S2,-0.968061,0.012693,2.516691,Na5LiS2
4,Li3Hg8S5,-0.395427,0.193329,0.854263,Li3Hg8S5
...,...,...,...,...,...
301,Li2Sn15S19,-0.768460,0.005445,0.757977,Li2Sn15S19
302,Li6Ga1S4,-1.404617,-0.001322,2.905212,Li6GaS4
303,Li17Pb6S5,-0.763350,0.172872,0.511936,Li17Pb6S5
304,Ba15Li2S19,-2.013019,0.103946,2.870701,Ba15Li2S19


In [8]:
merged_result = pd.merge(screened_by_stability_window_df,df,on=['composition','pretty_formula'],how='inner')
merged_result.to_csv('./data/04_2_after_stability_window_screening.csv',index=False)

In [9]:
merged_result = pd.read_csv('./data/04_2_after_stability_window_screening.csv')
merged_result

,pretty_formula,composition,lower_bound,upper_bound,total_range,roost_ensemble_mean,e_hull_from_roost,roost_band_gap
0,Li6PbS4,Li6Pb1S4,1.471095,2.106273,0.635178,-1.355585,0.017395,1.111903
1,LiGaS2,Li1Ga1S2,1.304365,2.635792,1.331427,-1.281533,-0.004185,3.948630
2,Ba(LiS)2,Ba1Li2S2,-0.000000,1.955832,1.955832,-1.837303,-0.003044,3.472003
3,Li6ZnS4,Li6Zn1S4,1.192349,2.129135,0.936786,-1.424084,-0.000423,2.650219
4,Li6SnS4,Li6Sn1S4,1.557811,1.720368,0.162556,-1.296940,0.060274,1.300834
5,LiAl5S2,Li1Al5S2,0.965560,1.598013,0.632453,-0.571905,0.193821,2.928105
6,Sr(Li3S2)2,Sr1Li6S4,-0.000000,2.059910,2.059910,-1.678688,-0.012882,3.400787
7,LiS2O5,Li1S2O5,3.488995,3.495805,0.006809,-1.808928,0.184285,6.266000
8,Li2B6S,Li2B6S1,0.948059,1.586536,0.638477,-0.415476,0.085774,2.205275
9,LiAsS2,Li1As1S2,1.903691,2.225514,0.321823,-0.855857,0.003279,1.967993
